In [1]:
%matplotlib tk
%load_ext autoreload
%autoreload 2
import SIKE
import os
import numpy as np
import matplotlib.pyplot as plt
import sk_plotting_functions as spf

In [2]:
impzeff_opts = {'EVOLVE': True,
                'SAVE': False,
                'MODELLED SPECIES': ['C'],
                'DELTA_T':1.0e5, 
                'RES_THRESH': 1e-10, 
                'MAX_STEPS': 1e4, 
                'T_SAVE': 1e6, 
                'FRAC_IMP_DENS': 0.05, 
                'COLL_ION_REC': True, 
                'RAD_REC': True, 
                'COLL_EX_DEEX': True, 
                'SPONT_EM': True, 
                'GS_ONLY': False, 
                'GS_ONLY_RADREC': False,
                'COMPARE_ADAS': True}
P_in = 64
n_avg = 8
run_dir = '/Users/dpower/Documents/01 - PhD/14 - ELM investigation/01 - Runs/01 - Equilibria/02 - Kinetic/P_in = ' + str(P_in) + 'MW/Output_job_EQ_K' + str(P_in) + '_' + str(n_avg) + 'e19'
sktrun = spf.SKTRun(run_dir)

In [3]:
# Single ImpZeff run (SOLKIT)
sp = SIKE.run(sktrun=sktrun,opts=impzeff_opts)
impzeff_run = sp['C']


Initialising...


/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/input.py:236: RuntimeWarning: overflow encountered in exp
  (np.exp(y) * scipy.special.exp1(y)) * (B - (C * y) + (0.5 * D * y * y) + (E/y)))
/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/input.py:236: RuntimeWarning: invalid value encountered in double_scalars
  (np.exp(y) * scipy.special.exp1(y)) * (B - (C * y) + (0.5 * D * y * y) + (E/y)))


Building rate matrices...
Solving state densities...


In [12]:
# Single ImpZeff run (ARRAYS)
num_x = 250
Te = np.geomspace(0.1,100.0,num_x)
ne = np.geomspace(1e0,1e0,num_x)
sp = SIKE.run(Te=Te,ne=ne,opts=impzeff_opts)
impzeff_run = sp['C']


Initialising...
Building rate matrices...


/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/input.py:236: RuntimeWarning: overflow encountered in exp
  (np.exp(y) * scipy.special.exp1(y)) * (B - (C * y) + (0.5 * D * y * y) + (E/y)))
/Users/dpower/Documents/01 - PhD/01 - Code/08 - SIKE/input.py:236: RuntimeWarning: invalid value encountered in double_scalars
  (np.exp(y) * scipy.special.exp1(y)) * (B - (C * y) + (0.5 * D * y * y) + (E/y)))


Solving state densities...


In [13]:
impzeff_run.plot_dens(xaxis='Te',compare_adas=True,logx=True,plot_kin=False,normalise=True)
# impzeff_run.plot_Zavg(compare_adas=True,plot_saha=False,logx=True,xaxis='Te')
# impzeff_run.plot_ion_rates(compare_adas=True,plot_sk=False)
# impzeff_run.plot_PLT(xaxis='Te',compare_adas=True,plot_stages=True,plot_eff=False,plot_sk=False,logx=True)
# impzeff_run.plot_radiation(compare_adas=False, plot_sk=True, plot_max=True)

In [357]:
fig,ax=plt.subplots(1)
rr, _ = impzeff_run.get_radrec_E_rates(per_z=True)
for z in range(7):
    ax.plot(impzeff_run.Te*impzeff_run.Te, rr[:,z],label=str(z))
    ax.legend()

In [371]:
fig,ax = plt.subplots(1)
dens = impzeff_run.dens
zdens, _, _ = impzeff_run.get_z_dens()
x = impzeff_run.Te*impzeff_run.T_norm
# x = impzeff_run.xgrid
for state in impzeff_run.states:
    if state.iz == 0:
    # if state.iz == 0 and state.statename=='2s2,2p1,3d1 1F':
    # if state.iz == 0 and state.statename=='2s2,2p2 1D':
        ax.plot(x,dens[:,state.loc]/zdens[:,0])
# ax.plot(x,zdens[:,0],'--',color='black')
ax.set_yscale('log')

In [370]:
fig,ax = plt.subplots(1)
for i in range(20,len(impzeff_run.iz_transitions)):
    if impzeff_run.iz_transitions[i].from_state.iz == 0:
        impzeff_run.iz_transitions[i].plot_cs(ax=ax,logx=True,logy=True)

In [236]:
# impzeff_run.plot_ion_rates(compare_adas=True)
impzeff_run.plot_rec_rates(compare_adas=True)

In [13]:
import aurora

adas_iz_file = aurora.adas_file('imp_data/Carbon/scd96_c.dat')
adas_iz_file.plot()

In [109]:
import rates
fig,ax = plt.subplots(1)
i = 6
Te = 100 / impzeff_run.T_norm
trans = impzeff_run.iz_transitions[i]
grat = trans.to_state.statw / trans.from_state.statw
iz_sigma = trans.sigma 
tbrec_sigma = rates.sigma_tbrec(iz_sigma,grat,impzeff_run.vgrid/impzeff_run.v_th,Te,eps)
Egrid = impzeff_run.T_norm * (impzeff_run.vgrid / impzeff_run.v_th) ** 2
ax.plot(Egrid,iz_sigma)
ax.plot(Egrid,tbrec_sigma,'--')
ax.set_yscale('log')

IndexError: list index out of range

In [307]:
fig,ax = plt.subplots(1)

In [309]:
# Calculate ionization rate coefficient as function of T_e
import rates
ir = impzeff_run
trans = ir.iz_transitions[18]

sigma = trans.sigma
label = str(trans.from_state.iz) + '+  ' + trans.from_state.statename + \
            ' -> ' + str(trans.to_state.iz) + \
            '+ ' + trans.to_state.statename + ''


Te = np.geomspace(1.0,100,100) / ir.T_norm
ne = 1e21 / ir.n_norm
rate_coeffs = np.zeros(100)
collrate_const = ir.n_norm * ir.v_th * ir.sigma_0 * ir.t_norm
for i in range(100):
    f0 = spf.maxwellian(Te[i],ne,ir.vgrid/ir.v_th)
    
    ne_loc = 4 * np.pi * np.sum(f0 * ((ir.vgrid/ir.v_th)**2) * (ir.dvc/ir.v_th))
    
    rate_coeffs[i] = collrate_const * rates.ion_rate(f0,ir.vgrid/ir.v_th,ir.dvc/ir.v_th,sigma) / ne_loc
    rate_coeffs[i] /= (ir.t_norm * ir.n_norm)

ax.plot(Te*ir.T_norm, 1e6*rate_coeffs,label=label)

ax.set_yscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Rate coeff')
ax.set_xscale('log')
ax.grid()
ax.legend()

In [310]:
fig,ax = plt.subplots(1)

In [312]:
# Calculate three-body recombination rate coefficient as function of T_e
import rates
ir = impzeff_run

trans = ir.iz_transitions[18]
iz_sigma = trans.sigma
label = str(trans.from_state.iz) + '+  ' + trans.from_state.statename + \
            ' -> ' + str(trans.to_state.iz) + \
            '+ ' + trans.to_state.statename + ''

eps = trans.thresh / ir.T_norm
grat = trans.to_state.statw / trans.from_state.statw

Te = np.geomspace(1.0,100,100) / ir.T_norm
ne = 1e19 / ir.n_norm
rate_coeffs = np.zeros(100)
collrate_const = ir.n_norm * ir.v_th * ir.sigma_0 * ir.t_norm
tbrec_norm = ir.n_norm * np.sqrt((spf.planck_h ** 2) / (2 * np.pi *
                                                                  spf.el_mass * ir.T_norm * spf.el_charge)) ** 3
for i in range(100):
    f0 = spf.maxwellian(Te[i],ne,ir.vgrid/ir.v_th)
    
    ne_loc = 4 * np.pi * np.sum(f0 * ((ir.vgrid/ir.v_th)**2) * (ir.dvc/ir.v_th))
    
    rate_coeffs[i] = collrate_const * tbrec_norm * rates.tbrec_rate(f0,Te[i],ir.vgrid/ir.v_th,ir.dvc/ir.v_th,eps,grat,iz_sigma) / ne_loc
    rate_coeffs[i] /= (ir.t_norm * ir.n_norm)

ax.plot(Te*ir.T_norm, 1e6*rate_coeffs,label=label)

# #Compare adas
# adas_rec_coeffs = ir.get_rec_rates_adas()
# adas_iz_coeffs = ir.get_ion_rates_adas()
# ax.plot(Te*ir.T_norm, 1e6*adas_rec_coeffs[:,0])

ax.set_yscale('log')
ax.set_xlabel('$T_e$ [eV]')
ax.set_ylabel('Rate coeff')
ax.set_xscale('log')
ax.grid()
ax.legend()

In [266]:
ir=impzeff_run
ion_rates, _ = ir.get_ion_rates()
ion_rates_adas = ir.get_ion_rates_adas()
zdens,_,_ = ir.get_z_dens()
rec_rates,_ = ir.get_tbrec_rates()
rec_rates_adas = ir.get_rec_rates_adas()
z = 5
fig,ax = plt.subplots(1)
ax.plot(ir.Te*ir.T_norm,rec_rates[:,z]*((ir.ne)**2)*zdens[:,z+1],'-',label='S_rec')
# ax.plot(ir.Te*ir.T_norm,rec_rates_adas[:,z]*((ir.ne)**2)*zdens[:,z+1], '--', label='S_rec adas')
ax.plot(ir.Te*ir.T_norm,ion_rates[:,z]*ir.ne*zdens[:,z],label='S_ion')
# ax.plot(ir.Te*ir.T_norm,ion_rates_adas[:,z]*ir.ne*zdens[:,z], '--', label='S_ion adas')
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid()